Summary Tables

In [2]:
# importing libraries
library(tidyverse)
library(repr)
library(janitor)
library(ggplot2)
library(tidymodels)
library(RColorBrewer)


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘janitor’


The following objects are masked from ‘package:stats’:

    chisq.test, fisher.test


── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     

── Conflicts ──────

In [3]:
# Cleaning names and specifying categorical variable
diabetes <- read_csv("../data/diabetes.csv") |>
        	clean_names() |>
        	mutate(outcome = as_factor(outcome)) |>
        	mutate(diabetes = fct_recode(outcome, "Yes" = "1", "No" = "0")) |>
        	select(-outcome)


Rows: 768 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (9): Pregnancies, Glucose, BloodPressure, SkinThickness, Insulin, BMI, D...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [4]:
# Displaying the data
head(diabetes)


pregnancies,glucose,blood_pressure,skin_thickness,insulin,bmi,diabetes_pedigree_function,age,diabetes
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
6,148,72,35,0,33.6,0.627,50,Yes
1,85,66,29,0,26.6,0.351,31,No
8,183,64,0,0,23.3,0.672,32,Yes
1,89,66,23,94,28.1,0.167,21,No
0,137,40,35,168,43.1,2.288,33,Yes
5,116,74,0,0,25.6,0.201,30,No


In [5]:
# splitting the diatabes data
diabetes_split <- initial_split(diabetes, prop = 0.75, strata = diabetes)

# training data
diabetes_training <- training(diabetes_split)

# testing data
diabetes_testing <- testing(diabetes_split)

# the data to be used
head(diabetes_training)


pregnancies,glucose,blood_pressure,skin_thickness,insulin,bmi,diabetes_pedigree_function,age,diabetes
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,85,66,29,0,26.6,0.351,31,No
1,89,66,23,94,28.1,0.167,21,No
5,116,74,0,0,25.6,0.201,30,No
10,115,0,0,0,35.3,0.134,29,No
4,110,92,0,0,37.6,0.191,30,No
10,139,80,0,0,27.1,1.441,57,No


In [7]:
#Determining the average values for those with and without diabetes
average_summary <- diabetes_training |>
group_by(diabetes) |>
summarize(average_glucose = mean(glucose),
      	average_bmi = mean(bmi),
     	average_blood_pressure = mean(blood_pressure))
average_summary


diabetes,average_glucose,average_bmi,average_blood_pressure
<fct>,<dbl>,<dbl>,<dbl>
No,108.5387,30.22133,67.60000
Yes,140.7761,35.70299,71.22388


In [8]:
#determining the number of data for each pregnancy stage
count_preg <- diabetes_training |>
count(pregnancies)
count_preg


pregnancies,n
<dbl>,<int>
0,84
1,106
2,67
3,53
4,54
5,48
6,38
7,31
8,28


In [10]:
#Average glucose, blood pressure, bmi levels for number of pregnancies considered as a factor
diabetes_training_pregnant_stage <- diabetes_training |>
mutate(pregnancies = as_factor(pregnancies)) |>
group_by(pregnancies) |>
summarize(
	average_glucose = mean(glucose),
	average_bmi = mean(bmi),
	average_blood_pressure = mean(blood_pressure))
diabetes_training_pregnant_stage


pregnancies,average_glucose,average_bmi,average_blood_pressure
<fct>,<dbl>,<dbl>,<dbl>
0,120.5119,34.01071,65.82143
1,112.2642,31.72264,68.05660
2,111.3881,31.04627,61.98507
3,119.5660,29.98113,66.83019
4,126.0370,32.19444,69.11111
5,119.9583,33.16875,76.54167
6,119.2632,30.06053,68.50000
7,134.8387,33.34516,67.83871
8,129.2500,32.47857,75.07143


### Expected Outcomes and Significance

What we would like to find:
- How does blood pressure impact the likelihood of diabetes?
- What is the relationship of BMI and diabetic standing?
- What is the relationship between Glucose and Age with an emphasis on their Diabetic Standing?


Such findings will be able to create a much bigger impact in the future, such as …


1. Finding which variables in the data set have the most significant impact on predicting diabetes in this dataset. (The problem with this is that we are going to be standardizing our data, so I am not sure if we will be able to determine the most impactful variable)
2. Determine what aspects/variables related to diabetes medical workers need to focus on to prevent/decrease the risk of future patients being diagnosed with either Type 1 or 2 diabetes
3. Determining the critical age range where individuals are more susceptible to developing diabetes based on glucose levels

Other potential findings:
1. To predict if an individual is susceptible to either Type 1 diabetes or Type 2 diabetes based on current data.
2. Determine if the increase in pregnancies leads to a higher risk of either Type 1 or 2 diabetes.
3. If individuals with high blood pressure are of higher risk of being diagnosed with either Type 1 or 2 diabetes
4. Visualize if BMI levels influences the diagnosis of Type 1 or 2 diabetes
